In [1]:
import os
import psycopg2
from dotenv import load_dotenv
load_dotenv()

rds_host=os.getenv('DB_HOST2')
rds_port = os.getenv('DB_PORT2')
db_name=os.getenv('DB_NAME2')
db_user=os.getenv('DB_USER2')
db_password=os.getenv('DB_PASS2')


In [2]:
# Establish a connection to your PostgreSQL database
connection = psycopg2.connect(
    host=rds_host,
    port=rds_port,
    database=db_name,
    user=db_user,
    password=db_password
)

In [3]:
# Create a cursor object to interact with the database
cursor = connection.cursor()

In [4]:
# Define SQL queries to create the tables

# 1. Create the State table
create_state_table = """
CREATE TABLE "state_table" (
    "area" int   NOT NULL,
    "area_title" varchar(30)   NOT NULL,
    "prim_state" varchar(30)   NOT NULL,
    CONSTRAINT "pk_State_Table2" PRIMARY KEY (
        "area"
     )
);
"""

In [5]:
# 2. Create the Occupation table
create_occupation_table = """
CREATE TABLE "occupation_table" (
    "occ_code" varchar(20)   NOT NULL,
    "occ_title" varchar(200)   NOT NULL,
    "o_group" varchar(40)   NOT NULL,
    CONSTRAINT "pk_Occupation_Table2" PRIMARY KEY (
        "occ_code"
     )
);
"""

In [6]:
# 3. Create the EmploymentWage table
create_employment_wage_table = """
CREATE TABLE "employmentwage_table" (
    "area" int   NOT NULL,
    "occ_code" varchar(10)   NOT NULL,
    "tot_emp" numeric(12,2)   NOT NULL,
    "emp_prse" numeric(12,2)   NOT NULL,
    "jobs_1000" numeric(12,2)   NOT NULL,
    "loc_quotient" numeric(12,2)   NOT NULL,
    "h_mean" numeric(12,2)   NOT NULL,
    "a_mean" numeric(12,2)   NOT NULL,
    "mean_prse" numeric(12,2)   NOT NULL,
    "h_pct25" numeric(12,2)   NOT NULL,
    "h_median" numeric(12,2)   NOT NULL,
    "h_pct75" numeric(12,2)   NOT NULL,
    "a_pct25" numeric(12,2)   NOT NULL,
    "a_median" numeric(12,2)   NOT NULL,
    "a_pct75" numeric(12,2)   NOT NULL,
    CONSTRAINT "pk_EmploymentWage_Table2" PRIMARY KEY (
        "area","occ_code"
     )
);
"""

In [7]:
# Execute the queries to create the tables
cursor.execute(create_state_table)
cursor.execute(create_occupation_table)
cursor.execute(create_employment_wage_table)

# Commit the changes to the database
connection.commit()

print("Tables created successfully.")

Tables created successfully.


In [8]:
# Define file paths for the CSV files
state_csv = 'data/State_Table.csv'
occupation_csv = 'data/Occupation_Table.csv'
employment_wage_csv = 'data/EmploymentWage_Table.csv'

In [9]:
# Function to upload CSV 
def copy_csv_to_table(cursor, table_name, csv_file):
    try:
        with open(csv_file, 'r') as f:
            next(f)  # Skip the header row
            cursor.copy_expert(f"COPY {table_name} FROM stdin WITH CSV HEADER", f)
        print(f"Data loaded successfully into {table_name}")
    except Exception as e:
        print(f"Error loading data into {table_name}: {e}")
        connection.rollback()  # Rollback the transaction if there's any error
    else:
        connection.commit()  # Commit the transaction only if no error occurs

# loading data into the tables

try:
    # Upload CSV data into the State_Table2
    copy_csv_to_table(cursor, "state_table", state_csv)

    # Upload CSV data into the Occupation_Table2
    copy_csv_to_table(cursor, "occupation_table", occupation_csv)

    # Upload CSV data into the EmploymentWage_Table2
    copy_csv_to_table(cursor, "employmentwage_table", employment_wage_csv)
except Exception as e:
    print(f"Transaction failed: {e}")
finally:
    cursor.close()
    connection.close()

Data loaded successfully into state_table
Data loaded successfully into occupation_table
Data loaded successfully into employmentwage_table
